In [ ]:
%matplotlib inline
import pandas as pd
import rpy2,os,re
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
import seaborn as sns
sns.set(font="Arial")
sns.set_style("white")
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
from sklearn.decomposition import PCA as sklearnPCA
import seaborn as sns
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import pickle
from statsmodels.sandbox.stats.multicomp import multipletests


from scipy.stats import ttest_ind,ttest_rel, zscore, spearmanr, pearsonr
from statannot import add_stat_annotation
from scipy.interpolate import interp1d
import gseapy as gp
from matplotlib.gridspec import GridSpec
from scipy.spatial import distance
from scipy.cluster.hierarchy import dendrogram, linkage, cut_tree

## Load File (Protein and clinical Individual Data)

In [ ]:
temp=pd.ExcelFile('Data/ProteomicsData.xlsx')

data=temp.parse('all data',index_col=0)
data=data[data.columns[data.columns.str.contains('tmt10plex')]]
data.columns=[i.split('_')[2]+'_'+i.split('_')[3] for i in data.columns]
metadata=temp.parse('Labeling',index_col=0)
metadata=metadata[metadata['Outliers'] == 'NO']
metadata['Condition']=metadata['Condition'].str.replace(' ','')
intersect=set(data.columns).intersection(metadata.index)
metadata=metadata.reindex(intersect)
data=data[intersect]
mapping=temp.parse('Mapping',index_col=0)

data_clin = pd.read_excel('Data/ClinicalVariables.xlsx',index_col = 0).T

## Load File (Protein Statistical Analysis Results)

The statistical analyses were done using DEQms package in R. Code will be uploaded in a separate file

https://www.bioconductor.org/packages/release/bioc/html/DEqMS.html

In [ ]:
#Loading deqms results
dep_dict = {}
for z in os.listdir('Results/deqms/'):
    j,i=z.replace('.txt','').replace('deqms_','').split('vs')
    temp=pd.read_csv('Results/deqms/deqms_%svs%s.txt' % (j,i),sep='\t',index_col=0)[['name','logFC','sca.adj.pval']].sort_values('sca.adj.pval')
    temp.columns=['Gene Name', 'LogFC', 'FDR (deqms)']
    temp['Direction'] = ['DOWN' if k < 0 else 'UP' for k in temp['LogFC']]
    temp.loc[temp['FDR (deqms)'].isna(),'Direction'] = np.nan
    dep_dict['%sVS%s' % (j,i)] = temp

## Load File (KEGG Enrichment Analysis Results per comparison)

In [ ]:
enrichment_dict = {}

for z in os.listdir('Results/KEGG/'):
    j,i=z.replace('.txt','').replace('piano_','').split('vs')
    piano=pd.read_csv('Results/KEGG/piano_%svs%s.txt' % (j,i),sep='\t',index_col='Name')
    up=piano[piano['p adj (dist.dir.up)'] < 0.05][['Genes (tot)', 'Stat (dist.dir.up)','p adj (dist.dir.up)']]
    up['DIR']='UP'
    up.columns=['# Genes', 'Stats', 'P-Adj','Direction']
    dn=piano[piano['p adj (dist.dir.dn)'] < 0.05][['Genes (tot)', 'Stat (dist.dir.up)','p adj (dist.dir.dn)']]
    dn['DIR']='DOWN'
    dn.columns=['# Genes', 'Stats','P-Adj','Direction']
    enrichment_dict['%sVS%s' % (j,i)] = pd.concat([up,dn])

## Load File (RNA-Seq Statistical Analysis)

https://pubmed.ncbi.nlm.nih.gov/32579934/

In [ ]:
# Loading Transcriptomics Data Analysis Results from https://pubmed.ncbi.nlm.nih.gov/32579934/
# Download from https://ars.els-cdn.com/content/image/1-s2.0-S2211124720307890-mmc2.xlsx (supplemental material)
temp = pd.ExcelFile('Results/1-s2.0-S2211124720307890-mmc2.xlsx')

degs = pd.DataFrame()
for i in temp.sheet_names:
    degs = pd.concat([degs, temp.parse(i,index_col = 0)['Direction'].rename(i)], axis = 1)

## Figure 1C

In [ ]:
var = 'Condition'
sklearn_pca = sklearnPCA(n_components=2)
Y_sklearn = sklearn_pca.fit_transform(data.dropna().T)
plt.figure(figsize=(6,5))
coordinate = pd.DataFrame(Y_sklearn, columns = ['x', 'y'], index = metadata[var]).reset_index()
x = []
for i in coordinate[var]:
    if 'Control' in i:
        x.append('Control')
    if 'Strength' in i:
        x.append('Strength')
    if 'Endurance' in i:
        x.append('Endurance')
coordinate['style'] = x
coordinate['color'] = ['Male' if 'Male' in i else 'Female' for i in coordinate[var]]
ax = sns.scatterplot(data = coordinate, x = 'x', y = 'y', s = 300, hue = 'color', style = 'style')
ax.set_xlabel('PC1 (%.2f%%)' % (sklearn_pca.explained_variance_ratio_[0]*100), fontsize = 20, fontweight = 'bold')
ax.set_ylabel('PC2 (%.2f%%)' % (sklearn_pca.explained_variance_ratio_[1]*100), fontsize = 20, fontweight = 'bold')
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.get_legend().set_title('Groups')
plt.setp(ax.get_legend().get_title(), fontsize='15', fontweight = 'bold')
plt.setp(ax.get_legend().get_texts(), fontsize='15')
plt.legend(ncol=2)
plt.tight_layout()

## Figure 1D

Figure 1D in the paper has undergone a post-processing in Illustrator to combine multiple smaller subgroups to "Others" category.

In [ ]:
## Loading The direction of differentially expressed proteins
dep = pd.DataFrame()
for i in dep_dict.keys():
    temp = dep_dict[i]
    temp = temp[temp['FDR (deqms)'] < 0.05]['Direction'].rename(i)
    dep = pd.concat([dep, temp],1)

In [ ]:
#ordering based on the number of DEPs members
legend_order = list(temp2_color.value_counts().sort_values(ascending = False).index)

#mapping the DEPs group to colors
temp = dep.unstack().reset_index().dropna()
temp2 = temp.groupby('level_1')['level_0'].apply(lambda x: '; '.join(sorted(set(x))))
cmap = dict(zip(temp2.unique(), sns.color_palette('tab20').as_hex()))

temp_num = pd.DataFrame(temp2).reset_index()
temp_num['col'] = temp_num['level_0'].replace(cmap)
temp_num = temp_num.groupby(['col','level_0']).count().reset_index().set_index('col').reindex(legend_order)
temp_num['Legend'] = temp_num['level_0'] + ' (' + temp_num['level_1'].astype(str) + ' DEPs)'

In [ ]:
#generating the legends
colors = temp_num.index.tolist()
f = lambda m,c: plt.plot([],[],marker=m, color=c, ls="none")[0]
handles = [f("s", colors[i]) for i in range(temp_num.shape[0])]
labels = temp_num['Legend']
legend = plt.legend(handles, labels, loc=3, framealpha=1, frameon=True)

def export_legend(legend, filename="../221014/legend2.pdf", expand=[-5,-5,5,5]):
    fig  = legend.figure
    fig.canvas.draw()
    bbox  = legend.get_window_extent()
    bbox = bbox.from_extents(*(bbox.extents + np.array(expand)))
    bbox = bbox.transformed(fig.dpi_scale_trans.inverted())

export_legend(legend)
plt.show()

In [ ]:
fin = []
temp = temp.replace(temp2.replace(cmap))
for i in temp['level_0'].unique():
    temp3 = pd.DataFrame(temp[temp['level_0'] == i].groupby('level_1')[0].count().reindex(legend_order).dropna()).cumsum().reset_index()
    temp3['comp'] = i
    fin.append(temp3)
fin = pd.concat(fin)

In [ ]:
fig, axs = plt.subplots(nrows = 3, ncols = 1, figsize = (4, 6), gridspec_kw={'height_ratios':[2,4,2]}, sharex = True)
g = axs[2]
for i in range(fin.shape[0])[::-1]:
    temp = pd.DataFrame(fin.reset_index().iloc[i]).T
    sns.barplot(data = temp, x = 'comp', y = 0, palette = temp['level_1'], 
                order = ['EnduranceMaleVSControlMale', 'EnduranceFemaleVSControlFemale', 'StrengthMaleVSControlMale', 'EnduranceMaleVSStrengthMale','ControlFemaleVSControlMale', 'EnduranceFemaleVSEnduranceMale'], 
                ax = g)
g.set_ylim(0,30)
sns.despine(ax = g, top = True, bottom = False, right = False, left = False)
g.set_xticklabels(['MEvsMC', 'FEvsFC', 'MSvsMC', 'MEvsMS', 'FCvsMC', 'FEvsFC'], rotation = 30, ha = 'right')
g.set_ylabel('')
g.set_xlabel('')

g = axs[1]
for i in range(fin.shape[0])[::-1]:
    temp = pd.DataFrame(fin.reset_index().iloc[i]).T
    sns.barplot(data = temp, x = 'comp', y = 0, palette = temp['level_1'], 
                order = ['EnduranceMaleVSControlMale', 'EnduranceFemaleVSControlFemale', 'StrengthMaleVSControlMale', 'EnduranceMaleVSStrengthMale','ControlFemaleVSControlMale', 'EnduranceFemaleVSEnduranceMale'], 
                ax = g)
g.set_ylim(350,700)
sns.despine(ax = g, top = True, bottom = True, right = False, left = False)
g.set_xticklabels(['MEvsMC', 'FEvsFC', 'MSvsMC', 'MEvsMS', 'FCvsMC', 'FEvsFC'], rotation = 30, ha = 'right')
g.set_ylabel('')
g.set_xlabel('')


g = axs[0]
for i in range(fin.shape[0])[::-1]:
    temp = pd.DataFrame(fin.reset_index().iloc[i]).T
    sns.barplot(data = temp, x = 'comp', y = 0, palette = temp['level_1'], 
                order = ['EnduranceMaleVSControlMale', 'EnduranceFemaleVSControlFemale', 'StrengthMaleVSControlMale', 'EnduranceMaleVSStrengthMale','ControlFemaleVSControlMale', 'EnduranceFemaleVSEnduranceMale'], 
                ax = g)
g.set_ylim(850,900)
sns.despine(ax = g, top = False, bottom = True, right = False, left = False)
g.set_xticklabels(['MEvsMC', 'FEvsFC', 'MSvsMC', 'MEvsMS', 'FCvsMC', 'FEvsFC'], rotation = 30, ha = 'right')
g.set_ylabel('')
g.set_xlabel('')

## Figure 2A-B

Created with R EnhancedVolcano Package based on the files in "Results/deqms/""

## Figure 2C-D

In [ ]:
dep = pd.DataFrame()
lst = ['EnduranceFemaleVSControlFemale', 'EnduranceMaleVSControlMale']

for i in lst:
    temp = dep_dict[i]
    temp = temp[temp['FDR (deqms)'] < 0.05]['Direction'].rename(i)
    dep= pd.concat([dep, temp],1)

In [ ]:
#Filtering for common proteins in Endurance Male and Female
dep_endurance = dep[dep['EnduranceFemaleVSControlFemale'] == dep['EnduranceMaleVSControlMale']].sort_values(['EnduranceFemaleVSControlFemale', 'EnduranceMaleVSControlMale'])
dep_endurance.loc[dep_endurance['EnduranceFemaleVSControlFemale'] == 'DOWN', 'group'] = 'Both DOWN'
dep_endurance.loc[dep_endurance['EnduranceFemaleVSControlFemale'] == 'UP', 'group'] = 'Both UP'

In [ ]:
#Functional Analysis of Both UP and both DOWN in endurance
exclude_KEGG = ['Hepatitis B', 'Human T-cell leukemia virus 1 infection', 'Regulation of actin cytoskeleton', 'Measles',
           'MicroRNAs in cancer', 'Pathways in cancer', 'Influenza A', 'Oocyte meiosis', 'Progesterone-mediated oocyte maturation',
           'Ribosome biogenesis in eukaryotes', 'Amoebiasis', 'Small cell lung cancer', 'Epstein-Barr virus infection', 'Human papillomavirus infection',
           'Pertussis', 'Staphylococcus aureus infection', 'Proteoglycans in cancer', 'Leishmaniasis', 'Legionellosis', 'Viral carcinogenesis', 
           'Chagas disease (American trypanosomiasis)', 'Toxoplasmosis', 'Malaria', 'Metabolism of xenobiotics by cytochrome P450', 'Chronic myeloid leukemia',
           'Prion diseases', 'Inflammatory bowel disease (IBD)', 'Kaposi sarcoma-associated herpesvirus infection', 'Parkinson disease', 'Hepatitis C', 'Systemic lupus erythematosus',
           'Tuberculosis', 'Hepatitis B', 'Measles', 'Dilated cardiomyopathy (DCM)', 'Hematopoietic cell lineage', 'Adrenergic signaling in cardiomyocytes',
           'Epstein-Barr virus infection', 'Drug metabolism', 'Salivary secretion', 'Thermogenesis', 'Glioma', 'Human cytomegalovirus infection', 'Melanoma', 'Chemical carcinogenesis',
            'African trypanosomiasis', 'Collecting duct acid secretion', 'African trypanosomiasis', 'Vascular smooth muscle contraction', 'Fluid shear stress and atherosclerosis',
            'Rheumatoid arthritis', 'Allograft rejection'
          ]

terms = pd.DataFrame()

for direction in dep_endurance['group'].unique():
    gene_list = list(set([mapping['Gene Name'][i] for i in dep_endurance['group'][dep_endurance['group'] == direction].index]))
    enr = gp.enrichr(gene_list=gene_list,
                     gene_sets='KEGG_2019_Human',
                     background=data.shape[0],
                     outdir='test/enrichr_kegg2',
                     cutoff=0.5,
                     verbose=False)
    temp = enr.res2d[enr.res2d['Adjusted P-value'] < 0.05]
    temp['direction'] = direction
    temp['-Log10(FDR)'] = -np.log10(temp['Adjusted P-value'])
    temp['#ofGenes'] = [len(i.split(';')) for i in temp['Genes']]
    terms = pd.concat([terms, ])
    
terms = terms[~terms.index.isin(exclude_KEGG)]
terms = terms[~((terms.index.str.contains('isease')) |
         (terms.index.str.contains('arcinoma')) |
          (terms.index.str.contains('ancer')) |
        (terms.index.str.contains('nsulin')) |
        (terms.index.str.contains('iabet')) |
        (terms.index.str.contains('ysosome')) |
        (terms.index.str.contains('nfection')) |
        (terms.index.str.contains('ardio')) |
        (terms.index.str.contains('ardia')) |
        (terms.index.str.contains('mmunodef')))]

fig, axs = plt.subplots(
        nrows=len(dep_endurance['group'].unique()), ncols=1, sharex=False, sharey=False,  figsize = (3,6))


for i, direction in enumerate(terms.groupby('direction').count().index):
    temp = terms[terms['direction'] == direction].reset_index().sort_values('-Log10(FDR)', ascending = False).iloc[0:10]
    print(temp.shape[0])
    ax = sns.barplot(data = temp, y = 'Term', x = '-Log10(FDR)', ax = axs[i])
    ax.set_ylabel(direction)
    sns.despine(ax = ax)

## Figure 2E

## Figure 2F

In [ ]:
kegg = pd.DataFrame()
lst = ['EnduranceFemaleVSControlFemale', 'EnduranceMaleVSControlMale', 'StrengthMaleVSControlMale']

for i in lst:
    temp = enrichment_dict[i]
    temp = temp['Stats'].rename(i)
    kegg= pd.concat([kegg, temp],1)
    
kegg = kegg[~kegg.index.isin(exclude_KEGG)]
kegg = kegg[~((kegg.index.str.contains('isease')) |
         (kegg.index.str.contains('arcinoma')) |
          (kegg.index.str.contains('ancer')) |
        (kegg.index.str.contains('ardio')) |
            (kegg.index.str.contains('ardia')) |
        (kegg.index.str.contains('iabet')) |
        (kegg.index.str.contains('ysosome')) |
        (kegg.index.str.contains('nfection')) |
            (kegg.index.str.contains('mycin')) |
            (kegg.index.str.contains('astric')) |
        (kegg.index.str.contains('mmunodef')))]

In [ ]:
temp = kegg[
    (kegg['EnduranceMaleVSControlMale'].notna() & kegg['EnduranceFemaleVSControlFemale'].notna()) | (kegg['StrengthMaleVSControlMale'].notna())
][['EnduranceMaleVSControlMale', 'EnduranceFemaleVSControlFemale', 'StrengthMaleVSControlMale']].dropna(how = 'all')

cmap=matplotlib.colors.LinearSegmentedColormap.from_list("", ["#0000a5",'#0000d8',"#FFFAF0",'#d80000',"#a50000"])
g = sns.clustermap(temp.fillna(0), yticklabels = 1, 
               center = 0, cmap = cmap, vmin = -30, vmax = 30,
               col_cluster = False, row_cluster = True,
               figsize = (6,12),
               cbar_kws={"orientation": "vertical","shrink": 1, 'label': 'Gene-Set Statistics'}
              )

## Figure 2G

In [ ]:
kegg = pd.DataFrame()
lst = ['ControlFemaleVSControlMale', 'EnduranceFemaleVSEnduranceMale']

for i in lst:
    temp = enrichment_dict[i]
    temp = temp['Stats'].rename(i)
    kegg= pd.concat([kegg, temp],1)
    
kegg = kegg[~kegg.index.isin(exclude_KEGG)]
kegg = kegg[~((kegg.index.str.contains('isease')) |
         (kegg.index.str.contains('arcinoma')) |
          (kegg.index.str.contains('ancer')) |
        (kegg.index.str.contains('ardio')) |
            (kegg.index.str.contains('ardia')) |
        (kegg.index.str.contains('iabet')) |
        (kegg.index.str.contains('ysosome')) |
        (kegg.index.str.contains('nfection')) |
            (kegg.index.str.contains('mycin')) |
            (kegg.index.str.contains('astric')) |
        (kegg.index.str.contains('mmunodef')))]

In [ ]:
temp = kegg.dropna(how = 'all')

cmap=matplotlib.colors.LinearSegmentedColormap.from_list("", ["#0000a5",'#0000d8',"#FFFAF0",'#d80000',"#a50000"])
g = sns.clustermap(temp.fillna(0), yticklabels = 1, 
               center = 0, cmap = cmap, vmin = -30, vmax = 30,
               col_cluster = False, row_cluster = True,
               figsize = (6,12),
               cbar_kws={"orientation": "vertical","shrink": 1, 'label': 'Gene-Set Statistics'}
              )

## Figure 3 and Network Genreation for Figure 4

Figure 3 and Network Generation for Figure 4 used protein, clinical, and transcriptomics data using the pipelines described in iNetModels (PMID: 33849075). Due to privacy reason, we cannot upload the transcriptomics data to GitHub, but it can be accessed through EGA Repository as described on our previous publication (PMID: 32579934)

## Figure 4A

Post-Processed in Illustrator for aesthetic and removing the values from top triangle

In [ ]:
corr = data_clin.T.corr(method = 'spearman')

In [ ]:
temp = spearmanr(data_clin.T, nan_policy='omit')
corr = pd.DataFrame(temp[0], index = data_clin.index, columns = data_clin.index)
pval = pd.DataFrame(temp[1], index = data_clin.index, columns = data_clin.index)
temp = pval.unstack().reset_index()
temp['padj'] = multipletests(temp[0],method='fdr_bh')[1]
padj = temp.pivot_table(index = 'level_0', columns = 'level_1', values = 'padj')

annot = padj[padj < 0.05].loc[corr.index, corr.columns]
annot = annot[corr != 1]
annot[annot.notna()] = '*'

In [ ]:
cmap=matplotlib.colors.LinearSegmentedColormap.from_list("", ["#0000a5",'#0000d8',"#FFFAF0",'#d80000',"#a50000"])
g = sns.clustermap(corr, xticklabels = 1, yticklabels = 1, cmap = cmap, center = 0, vmin = -1, vmax = 1, 
                   annot = corr, annot_kws={"size": 6},
                  )
g.ax_heatmap.set_xticklabels(g.ax_heatmap.get_xticklabels(), fontsize = 15)
g.ax_heatmap.set_yticklabels(g.ax_heatmap.get_yticklabels(), fontsize = 15)
g.ax_heatmap.tick_params(right=False, bottom=False)
mask = np.triu(np.ones_like(corr))
values = g.ax_heatmap.collections[0].get_array().reshape(corr.shape)
new_values = np.ma.array(values, mask=mask)
g.ax_heatmap.collections[0].set_array(new_values)

## Figure 4B

In [ ]:
## Loading The direction of differentially expressed proteins
dep = pd.DataFrame()
for i in dep_dict.keys():
    temp = dep_dict[i]
    temp = temp[temp['FDR (deqms)'] < 0.05]['Direction'].rename(i)
    dep = pd.concat([dep, temp],1)

In [ ]:
edges = pd.read_csv('Results/Network/edges.txt.gz',sep='\t')
nodes = pd.read_csv('Results/Network/nodes.txt',sep='\t',index_col = 0)

In [ ]:
top_genes = nodes[nodes['location'] == 'GENE']
top_genes = top_genes[top_genes['degree'] > top_genes['degree'].quantile(.99)]
#get only significant top genes
top_genes = top_genes[top_genes['symbol'].isin(degs.index)]

top_protein = nodes[nodes['location'] == 'PROTEIN']
top_protein = top_protein[top_protein['degree'] > top_protein['degree'].quantile(.99)]
#get only significant top proteins
top_protein = top_protein.reindex(set(top_protein.index).intersection(dep.index))

clinical_variables = nodes[nodes['location'] == 'CLINICAL']


central_and_clinical = pd.concat([top_genes, top_protein, clinical_variables])

In [ ]:
## Use the output (Results/Networks/Figure4B_*.txt) as the input of Cytoscape for the network visualization
edges[edges['source'].isin(central_and_clinical.index) & edges['target'].isin(central_and_clinical.index)].to_csv('Results/Network/Figure4B_edges.txt', sep = '\t')
central_and_clinical.to_csv('Results/Network/Figure4B_nodes.txt', sep = '\t')

## Figure 4C-D

Retrieved from InetModels and post-processed using Cytoscape

4C: https://inetmodels.com/?networkType=Multi-Omics+Network&categoryType=Study-specific+Networks&categoryName=Long-Term+Exercise+%28Emanuelsson+et+al+2023%29&analyteTypes=GENE%7C%2CPROTEIN&analytes=Vo2+Peak%7C%2CCS+activity&nodeLimit=30&pruning=0&firstNeighbour=true&visualize=true&correlation=both

4D: https://inetmodels.com/?networkType=Multi-Omics+Network&categoryType=Study-specific+Networks&categoryName=Long-Term+Exercise+%28Emanuelsson+et+al+2023%29&analyteTypes=GENE%7C%2CPROTEIN&analytes=Leg+Strength+%28Nm%29%7C%2CAnterior+Thigh+Volume+%28L%29&nodeLimit=30&pruning=0&firstNeighbour=true&visualize=true&correlation=both



## Figure 4E

Network figure was created in cytoscape based on the entire network and clustering information in the nodes and edges files in "Results/Network"

In [ ]:
for i in sorted(nodes['cluster'].unique()):
    gene_list = nodes[nodes['cluster'] == i]['symbol'].tolist()
    enr = gp.enrichr(gene_list=gene_list,
                     gene_sets='KEGG_2019_Human',
                     outdir='test/enrichr_kegg2',
                     cutoff=0.5,
                     verbose=False)
    temp = enr.res2d[enr.res2d['Adjusted P-value'] < 0.05]
    temp['-Log10(FDR)'] = -np.log10(temp['Adjusted P-value'])
    temp['#ofGenes'] = [len(i.split(';')) for i in temp['Genes']]
    terms = temp.set_index('Term').sort_values('-Log10(FDR)', ascending = False)[['-Log10(FDR)','#ofGenes','Genes']]

    terms = terms[~terms.index.isin(exclude_KEGG)]
    terms = terms[~((terms.index.str.contains('isease')) |
             (terms.index.str.contains('arcinoma')) |
              (terms.index.str.contains('ancer')) |
            (terms.index.str.contains('nsulin')) |
            (terms.index.str.contains('iabet')) |
            (terms.index.str.contains('ysosome')) |
            (terms.index.str.contains('nfection')) |
            (terms.index.str.contains('ardio')) |
            (terms.index.str.contains('ardia')) |
            (terms.index.str.contains('mmunodef')))]
    
    print('Cl-%d: %s' % (i, ', '.join(terms.index[0:10].tolist())))

## Figure 4E (Barplot)

In [ ]:
#getting # of each analyte in the clusters and their proportion (x100 to get hte percentage)
temp = ((nodes.groupby(['location', 'cluster'])['degree'].count()/nodes.groupby(['location'])['degree'].count())*100).reset_index()

In [ ]:
plt.figure(figsize = (3,3))
ax = sns.barplot(data = temp, x = 'location', y = 'degree', hue = 'cluster', palette = 'colorblind')
ax.legend_.remove()

In [ ]:
nodes.groupby(['location'])['degree'].count()

## Figure 5A

The main file ('Results/Figure5D-F.xlsx') was generated by using the accompanying R Code

https://elifesciences.org/articles/69802

In [ ]:
data_withGeneName = 2**(data.rename(index = mapping['Gene Name']).reset_index()).groupby('Protein ID').sum()
median_data = pd.concat([data_withGeneName.T, metadata['Condition']],1).groupby('Condition').median()

DEP_ALL = pd.ExcelFile('Results/Figure5D-F.xlsx')


In [ ]:
temp = pd.read_excel('Data/Figure5/elife-69802-supp2-v1.xlsx', index_col = 0).iloc[0:, 2:]
data_HIIT = temp.copy()
metadata_HIIT = pd.Series([i.split('_')[0] for i in temp.columns],index = temp.columns)
median_HIIT = pd.concat([data_HIIT.T, metadata_HIIT],1).groupby(0).median()

DEP = DEP_ALL.parse('MEMCvsHIIT')
median_HIIT = median_HIIT[set(DEP['Accession']).intersection(median_HIIT.columns)].rename(columns = DEP[['Accession', 'Gene Name']].set_index('Accession')['Gene Name'])#[DEP['Gene Name']]

prot_intersection = set(median_HIIT.columns).intersection(set(median_data.columns))#.intersection(DEP)

temp = pd.concat([zscore(median_data[prot_intersection].reindex()), zscore(median_HIIT[prot_intersection])]).T

x = []
for num, i in enumerate(temp.columns):
    for j in temp.columns[num+1:]:
        t1 = pearsonr(temp[i], temp[j])
        x.append([i, j, t1[0], t1[1]])
corr_HIIT = pd.DataFrame(x, columns = ['i','j', 'corr', 'pval'])
corr_HIIT['padj'] = multipletests(corr_HIIT['pval'],method='fdr_bh')[1]

corr_HIIT.to_csv('../Results/Corr_HIIT_Fig5A.txt', sep = '\t')

## Figure 5B

https://elifesciences.org/articles/49874

In [ ]:
temp = pd.ExcelFile('Data/Figure5/elife-49874-fig1-data3-v1.xlsx')
mapping_aging = pd.read_csv('Data/Figure5/mapping_prots_Aging.txt', sep = '\t', index_col = 0)
metadata_aging = pd.read_csv('Data/Figure5/metadata_Aging.txt', sep = '\t')
metadata_aging.index = metadata_aging['TMT Experiments'] + '_' + metadata_aging['TMT Channel'].astype(str)
data_aging = 2**(temp.parse('All-Proteins-5891-Quantified', index_col = 1, skiprows=4)[metadata_aging.index]).dropna().merge(mapping_aging, left_index = True, right_index = True).groupby('To').sum()
median_aging = pd.concat([data_aging.T, metadata_aging[['Age Level']]],1).groupby('Age Level').median()

In [ ]:
DEPs = DEP_ALL.parse('AllEndvsAging')['Gene Name']
prot_intersection = set(median_aging.columns).intersection(set(median_data.columns)).intersection(DEPs)

temp = pd.concat([zscore(median_data[prot_intersection].reindex()), zscore(median_aging[prot_intersection])]).T

x = []
for num, i in enumerate(temp.columns):
    for j in temp.columns[num+1:]:
        t1 = pearsonr(temp[i], temp[j])
        x.append([i, j, t1[0], t1[1]])
corr_Aging = pd.DataFrame(x, columns = ['i','j', 'corr', 'pval'])
corr_Aging['padj'] = multipletests(corr_Aging['pval'],method='fdr_bh')[1]

corr_Aging.to_csv('../Results/Corr_Aging_Fig5B.txt', sep = '\t')

## Figure 5C

In [ ]:
temp = pd.ExcelFile('Data/Figure5/1-s2.0-S2666379122003184-mmc2.xlsx')
metadata_T2D = temp.parse('Table S1a', index_col = 0)['T2D-3C']

data_T2D = 2**temp.parse('Table S1c', index_col = 0).dropna(how = 'all')[metadata_T2D.index]
median_T2D = pd.concat([data_T2D.T, metadata_T2D],1).groupby('T2D-3C').median()

In [ ]:
DEPs = DEP_ALL.parse('AllEndvsAging')['Gene Name']
prot_intersection = set(median_T2D.columns).intersection(set(median_data.columns)).intersection(DEPs)

temp = pd.concat([zscore(median_data[prot_intersection].reindex()), zscore(median_T2D[prot_intersection])]).T

x = []
for num, i in enumerate(temp.columns):
    for j in temp.columns[num+1:]:
        t1 = pearsonr(temp[i], temp[j])
        x.append([i, j, t1[0], t1[1]])
corr_T2DM = pd.DataFrame(x, columns = ['i','j', 'corr', 'pval'])
corr_T2DM['padj'] = multipletests(corr_T2DM['pval'],method='fdr_bh')[1]

corr_T2DM.to_csv('../Results/Corr_T2DM_Fig5C.txt', sep = '\t')

## Figure 5D-F

Refer to Figure5_DEP_ALL_generation.R